In [1]:
import os
import pandas as pd

In [4]:
res_pth = r"C:\Users\pechen\Documents\XTMF\Projects\407 Model Update\407 Model - Full Run - MCSpatial Run 33 - 20210607\Microsim Results"
zone_pth = r"C:\Users\pechen\Documents\407 Model Update"

zone_map = pd.read_csv(os.path.join(zone_pth,"407_ZoneMapping_spicy.csv"))
tripsmode_df = pd.read_csv(os.path.join(res_pth,"trip_modes.csv"))
trips_df = pd.read_csv(os.path.join(res_pth,"trips.csv"))
persons_df = pd.read_csv(os.path.join(res_pth,"persons.csv"))

In [5]:
trips_df = trips_df.merge(persons_df, on = ['household_id', 'person_id'])
trips_df = trips_df[trips_df.age >= 11]
trips_df = trips_df.merge(zone_map, left_on = 'o_zone', right_on = 'GTA06').drop(columns = "GTA06").rename(columns = {'region':'region_o'})
trips_df = trips_df.merge(zone_map, left_on = 'd_zone', right_on = 'GTA06').drop(columns = "GTA06").rename(columns = {'region':'region_d'})

In [6]:
trips_time_df = tripsmode_df.merge(trips_df, how = 'right', on = ['household_id', 'person_id', 'trip_id'])
trips_time_df["weight"] = trips_time_df["weight"]/10
trips_time_df["weight_z"] = trips_time_df["weight"]*trips_time_df["weight_x"]

In [7]:
am_trips_df = trips_time_df[(trips_time_df.o_depart >= 360) & (trips_time_df.o_depart < 540)]
pm_trips_df = trips_time_df[(trips_time_df.o_depart >= 900) & (trips_time_df.o_depart < 1140)]
ev_trips_df = trips_time_df[(trips_time_df.o_depart >= 1140) & (trips_time_df.o_depart < 1440)]

In [8]:
am_trips = pd.pivot_table(am_trips_df, index = ['region_o', 'region_d'], columns = 'mode', values = 'weight_z', aggfunc = sum, margins = True).fillna(0)
pm_trips = pd.pivot_table(pm_trips_df, index = ['region_o', 'region_d'], columns = 'mode', values = 'weight_z', aggfunc = sum, margins = True).fillna(0)
ev_trips = pd.pivot_table(ev_trips_df, index = ['region_o', 'region_d'], columns = 'mode', values = 'weight_z', aggfunc = sum, margins = True).fillna(0)

In [9]:
am_trips['drive'] = am_trips['Auto'] + am_trips['Passenger'] + am_trips['Carpool'] + am_trips['RideShare']
pm_trips['drive'] = pm_trips['Auto'] + pm_trips['Passenger'] + pm_trips['Carpool'] + pm_trips['RideShare']
ev_trips['drive'] = ev_trips['Auto'] + ev_trips['Passenger'] + ev_trips['Carpool'] + ev_trips['RideShare']

In [10]:
pm_out = pm_trips.filter(['Auto', 'All'])
ev_out = ev_trips.filter(['Auto', 'All'])
pm_out.to_csv(os.path.join(zone_pth, 'pm_out_30.csv'))
ev_out.to_csv(os.path.join(zone_pth, 'ev_out_30.csv'))